In [1]:
#!pip install dash_bootstrap_components

In [2]:
import os

In [3]:
cwd = os.getcwd()

In [4]:
import dash
from dash import dcc
from dash import html
from dash import dash_table

from collections import OrderedDict

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from dash import Dash, Input, Output, callback

from jupyter_dash import JupyterDash

import dash_bootstrap_components as dbc
COMPONENT_STYLE = "/assets/my_component.css"
external_stylesheets=[dbc.themes.BOOTSTRAP]

app = JupyterDash(__name__,requests_pathname_prefix="/dash1/",routes_pathname_prefix='/dash1/',
                  external_stylesheets=external_stylesheets,
                  meta_tags=[{'name': 'viewport', 'content': 'width=device-width, initial-scale=1'}],
                 suppress_callback_exceptions=True)
# Create server variable with Flask server object for use with gunicorn
server = app.server

In [5]:
from hello import hello_layout
#app.layout = hello_layout

In [6]:
from all_data_tables import years_table, limit_types_table
from all_data_tables import experiments_table, result_types_table, spin_dependency_table
from all_data_tables import greatest_hit_table, limits_table

from all_data_tables import years_df, limit_types_df
from all_data_tables import experiments_df, result_types_df, spin_dependency_df
from all_data_tables import greatest_hit_df, limits_df

from all_data_tables import limits_table_df

In [7]:
#app.layout = html.Div(['Hello World'])

#app.run_server(host="0.0.0.0", port=5051)

In [8]:
import pandas as pd

# Color Code

In [9]:
import plotly.graph_objects as go
import plotly.express as px
from itertools import cycle

# colors
palette = cycle(px.colors.qualitative.Bold)
#palette = cycle(['black', 'grey', 'red', 'blue'])

# SQL

In [10]:
experiments_df

,label,value
0,Theory,Theory
1,CDMS II (Soudan),CDMS II (Soudan)
2,CDMS I (SUF),CDMS I (SUF)
3,SuperCDMS,SuperCDMS
4,COSME,COSME
...,...,...
58,DRIFT,DRIFT
59,GAMBIT,GAMBIT
60,CDEX-10,CDEX-10
61,NEWS-G,NEWS-G


In [11]:
multiresult_df = limits_df[limits_df['id']==262]

multiresult_df

,id,spin_dependency,result_type,measurement_type,nomhash,x_units,y_units,x_rescale,y_rescale,default_color,...,rating,date_of_announcement,public,official,date_official,greatest_hit,date_of_run_start,date_of_run_end,year,rowid
244,262,SI,Th,Dir,b'--- \nknikolic: false\n',GeV,cm^2,,,DkG,...,0,2007-01-01,1,1,2009-07-16,0,None,None,2007.0,244


In [12]:
#multiresult_df.to_csv(cwd+'/data.csv')

In [13]:
#csv_df = pd.read_csv(cwd+'/data.csv')
#csv_df

In [14]:
# Define scale factors
def get_scale_factor(unit):
    BARN_CM2= 1e-24
    
    if (unit == "b"):
        return BARN_CM
    elif (unit == "mb"):
        return 1e-3*BARN_CM2
    elif (unit == "ub"):
        return 1e-6*BARN_CM2
    elif (unit == "nb"):
        return 1e-9*BARN_CM2
    elif (unit == "pb"):
        return 1e-12*BARN_CM2
    elif (unit == "fb"):
        return 1e-15*BARN_CM2
    elif (unit == "ab"):
        return 1e-18*BARN_CM2
    elif (unit == "zb"):
        return 1e-21*BARN_CM2
    elif (unit == "yb"):
        return 1e-24*BARN_CM2
    else: 
        return 1

In [15]:
data_df = multiresult_df

lol = []
for index, row in data_df.iterrows():
    #print(row['id'], row['data_values'])

    data_string = row[['data_values']].iloc[0]
    data_string = data_string.replace("{[", "")
    data_string = data_string.replace("]}", "")
    #print(data_string)
    data_series = data_string.split("]")
    #print(len(data_series))
    for l in range(0,len(data_series)):
        next_colour = next(palette)
        single_set = data_series[l]
        set_list = single_set.split(";")
        for i in set_list:
            z = i.split(" ");
            new_x = z[0].replace(",[", "")
            try:
                appendthis = [row['id'],l,new_x,z[1],next_colour]
            except:
                appendthis = [row['id'],l,0,0]
            lol.append(appendthis)
    #lol
df_experiment = pd.DataFrame(data=lol,columns=['experiment','series','raw_x','raw_y','series_color'])

unit = "zb" # this will probably be provided by the user via a drop-down menu
scale_factor = get_scale_factor(unit)


df_experiment['x'] = df_experiment['raw_x'].astype(str).astype(dtype = float, errors = 'ignore')

# add scale_factor here
df_experiment['y'] = df_experiment['raw_y'].astype(str).astype(dtype = float, errors = 'ignore')/scale_factor

   

In [16]:
import plotly.graph_objects as go
from plotly.validators.scatter.marker import SymbolValidator

raw_symbols = SymbolValidator().values
#raw_symbols

In [17]:
color_list = ['red','orange','yellow','lime green', 'green', 'blue-green', 'cyan',
              'sky blue', 'blue', 'purple', 'magenta', 'pink']

In [18]:
color_options=[{'label': i, 'value': i} for i in color_list]

In [19]:
line_styles_list = ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']

In [20]:
line_styles_options=[{'label': i, 'value': i} for i in line_styles_list]

In [21]:
symbol_list = ['circle','square','diamond','cross','x','hexagon','pentagon','octagon','star','asterisk','hash']

In [22]:
symbol_options=[{'label': i, 'value': i} for i in symbol_list]

In [23]:
#symbol_table = dash_table.DataTable(df.to_dict('records'), [{"name": i, "id": i} for i in df.columns])

In [27]:
df = pd.DataFrame(OrderedDict([
    ('experiment', ['1', '2', '3', '4']),
    ('color', ['red', 'green', 'blue', 'yellow']),
    ('line', ['solid', 'solid', 'solid', 'solid']),
    ('symbol', ['circle','square','diamond','cross']),
]))


format_table = html.Div([
    dash_table.DataTable(
        id='table-dropdown',
        row_deletable=True,
        # Add this line
        data=df.to_dict('records'),
        columns=[
            {'id': 'experiment', 'name': 'experiment'},
            {'id': 'color', 'name': 'color', 'presentation': 'dropdown'},
            {'id': 'line', 'name': 'line', 'presentation': 'dropdown'},
            {'id': 'symbol', 'name': 'symbol', 'presentation': 'dropdown'},
        ],

        editable=True,
        css=[{"selector": ".Select-menu-outer", "rule": "display: block !important"}],
        dropdown={
            'color': {
                'options': [
                    {'label': i, 'value': i}
                    for i in color_list
                ]
            },
            'line': {
                 'options': [
                    {'label': i, 'value': i}
                    for i in line_styles_list
                ]
            },
            'symbol': {
                 'options': [
                    {'label': i, 'value': i}
                    for i in symbol_list
                ]
            }
        }
    ),
    html.Div(id='table-dropdown-container')
])


In [28]:
color_dropdown = dcc.Dropdown(id='color-dropdown-id',options=color_options, value='red' )
line_dropdown = dcc.Dropdown(id='line-dropdown-id',options=line_styles_options, value='solid' )
symbol_dropdown = dcc.Dropdown(id='symbol-dropdown-id',options=symbol_options, value='diamond' )


app.layout = html.Div([
    color_dropdown,
    line_dropdown,
    symbol_dropdown,
    format_table,
    html.Div(id='dd-output-container')
])


#@app.callback(
#    Output('dd-output-container', 'children'),
#    Input('color-dropdown-id', 'value'),
#    #Input('line-dropdown-id', 'value'),
#    #Input('symbol-dropdown-id', 'value')
#)
#def update_output(chosen_color, chosen_line_style, chosen_symbol):
#    return f'You have selected {chosen_color}'##, chosen_line_style, chosen_symbol}'


In [29]:
app.run_server(host='0.0.0.0', port=5051)

Dash app running on http://0.0.0.0:5051/dash1/


/home/andrewcgaitskell/.local/share/virtualenvs/dmtoolnotes-z4lOQ0W-/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



In [ ]:
# Create figure
fig = go.Figure()

series_max = df_experiment['series'].max()

line_color=color_list[1]
line_style = line_styles[1]

# Loop df columns and plot columns to the figure
for i in range(1, series_max):
    trace2add = df_experiment[df_experiment['series']==i]
    fig.add_trace(go.Scatter(x=trace2add['x'], y=trace2add['y'],
                        mode='lines', # 'lines' or 'markers'
                        line=dict(width=4,dash=line_style,color=line_color),
                        #showscale=False,
                        name=i))
    fig.add_trace(go.Scatter(x=trace2add['x'], y=trace2add['y'],
                        mode='markers', # 'lines' or 'markers'
                        marker_symbol=symbol_list[1],
                             marker=dict(
                            size=10,
                            color=line_color,#set color equal to a variable
                            #colorscale='Viridis', # one of plotly colorscales
                            showscale=False,
                        ),
                        name=i))

fig.update_xaxes(type="log")
fig.update_yaxes(type="log")

fig.update_layout(showlegend=False)

fig.show()

In [ ]:
# Create figure
fig = go.Figure()


fig.add_trace(go.Scatter(x=df_experiment['x'], y=df_experiment['y'],
                        mode='lines+markers', ## lines' or 'markers'
                        marker=dict(
                            size=10,
                            color=df_experiment['series'],#set color equal to a variable
                            colorscale='Viridis', # one of plotly colorscales
                            showscale=True
                        ),
                         line=dict(color=df_experiment['series'],colorscale='Viridis', width=4,
                              dash='dash') 
                        ))

fig.update_xaxes(type="log")
fig.update_yaxes(type="log")


#fig = go.Figure(data=go.Scatter(
#    y = np.random.randn(500),
#    mode='markers',
#    marker=dict(
#        size=16,
#        color=np.random.randn(500), #set color equal to a variable
#        colorscale='Viridis', # one of plotly colorscales
#        showscale=True
#    )
#))

fig.show()

In [ ]:
#df_experiment[df_experiment['series']==7]['experiment'].unique()

In [ ]:
#df_experiment[df_experiment['series']==3]      

#df_plot = df_experiment[df_experiment['experiment']==262].copy() 
df_plot = df_experiment
#df_plot
fig = px.scatter(df_plot, x='x', y='y', color='series')

fig.update_xaxes(type="log")
fig.update_yaxes(type="log")

fig.show()
#new_df = dv_df.data_values.str.split(pat=']', n=- 1, expand=False)
#new_df.iloc[0]

# Drop Downs

In [ ]:
#app.run_server(mode='inline',host='0.0.0.0', port='5051')

#app.run_server(mode='inline', host="0.0.0.0", port=5051)

In [ ]:
dv = limits_df[limits_df['id']==100][['data_values']]
oneid = limits_df[limits_df['id']==100]

In [ ]:
allid = limits_df.copy()
#limits_df.columns
allid = allid[[
'id',
'data_values',         
'data_comment',
'data_label',
'data_reference',
'date_of_announcement',
'date_of_run_end',
'date_of_run_start',
'default_color',
'default_style',
'experiment',
'public',
'result_type',
'spin_dependency',
'x_rescale',
'x_units',
'y_rescale',
'y_units',
'year']]
allid.dtypes

In [ ]:
selected_id_set = [100]
    
#value_list = ["apple"]
boolean_series = allid.id.isin(selected_id_set)

filtered_df = allid[boolean_series]

filtered_df

In [ ]:
#plot_df.to_dict('records')
df = pd.DataFrame([[1, 1.5]], columns=['int', 'float'])

row = next(df.iterrows())[1]

type(row)

print(row)


In [ ]:
def datastring2dataframe(data_values_in):
    #try:
    #data_list = data_values_in['data_values'].tolist()
    if isinstance(data_values_in, pd.DataFrame):
        row_data = next(data_values_in.iterrows())[1]
    else:
        row_data = data_values_in
    data_string = row_data[['data_values']].iloc[0]
    data_string = data_string.replace("{[", "")
    data_string = data_string.replace("]}", "")
    x = data_string.split(";")
    lol = []
    for i in x:
        z = i.split(" ");
        appendthis = [z[0],z[1]]
        lol.append(appendthis)
    #lol
    df_experiment = pd.DataFrame(data=lol,columns=['raw_x','raw_y'])

    df_experiment['x'] = df_experiment['raw_x'].astype(str).astype(dtype = float, errors = 'ignore')
    df_experiment['y'] = df_experiment['raw_y'].astype(str).astype(dtype = float, errors = 'ignore')
    #except:
    #    data_null = [[0,0]]
    #    df_experiment = pd.DataFrame(data=data_null,columns=['raw_x','raw_y'])#

    #    df_experiment['x'] = df_experiment['raw_x'].astype(str).astype(float)
    #    df_experiment['y'] = df_experiment['raw_y'].astype(str).astype(float)
    #df_experiment.dtypes
    return df_experiment

In [ ]:
plot_df = allid
#plot_df = oneid

data_table = dash_table.DataTable(
        id='datatable-row-ids',
        columns=[
            {'name': i, 'id': i, 'deletable': True} for i in plot_df.columns
            # omit the id column
            if i not in ['data_comment','spin_dependency', 'measurement_type', 'nomhash',
       'x_units', 'y_units', 'x_rescale', 'y_rescale', 'default_color',
       'default_style', 'data_values', 'file_name',
        'created_at', 'updated_at',
       'creator_id', 'rating', 'date_of_announcement', 'public',
       'official', 'date_official', 'greatest_hit', 'date_of_run_start',
       'date_of_run_end']
        ],
        data=plot_df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode='multi',
        row_selectable='single',
        row_deletable=False,
        selected_rows=[],
        page_action='native',
        page_current= 0,
        page_size= 10,
        derived_virtual_selected_row_ids=[],
        selected_row_ids=[],
        style_cell = {'font_family': 'Arial','font_size': '12px',},
        style_data={'whiteSpace': 'normal','height': 'auto',},
        style_cell_conditional=[
            {'if': {'column_id': 'id'},
             'width': '2%'},
            {'if': {'column_id': 'data_label'},
             'width': '50%'},
            {'if': {'column_id': 'data_reference'},
             'width': '20%'},
            {'if': {'column_id': 'experiment'},
             'width': '10%'},
            {'if': {'column_id': 'year'},
             'width': '5%'},
        ],
    )

app.layout = html.Div([
    data_table,
    html.Div(id='datatable-row-ids-container')
])



@app.callback(
    Output(component_id='datatable-row-ids-container', component_property='children'),
    Input('datatable-row-ids', 'derived_virtual_row_ids'),
    Input('datatable-row-ids', 'selected_row_ids'),
    Input('datatable-row-ids', 'active_cell'))
def update_graphs(row_ids, selected_row_ids, active_cell):
    # When the table is first rendered, `derived_virtual_data` and
    # `derived_virtual_selected_rows` will be `None`. This is due to an
    # idiosyncrasy in Dash (unsupplied properties are always None and Dash
    # calls the dependent callbacks when the component is first rendered).
    # So, if `rows` is `None`, then the component was just rendered
    # and its value will be the same as the component's dataframe.
    # Instead of setting `None` in here, you could also set
    # `derived_virtual_data=df.to_rows('dict')` when you initialize
    # the component.
    
    #derived_virtual_data=data_df.to_dict('records')
    
    #selected_id_set = set(selected_row_ids or [])
    
    #value_list = ["apple"]

    boolean_series = plot_df.id.isin(selected_row_ids)

    filtered_df = plot_df[boolean_series].copy()
    
    #plot_df[plot_df['id']==100]

    plot_data = datastring2dataframe(filtered_df.iloc[0]).copy()
    
    if row_ids is None:
        dff = plot_data
        # pandas Series works enough like a list for this to be OK
        #row_ids = filtered_df['id']
        row_ids = [0]
    else:
        #dff = data_df.loc[row_ids]
        dff = plot_data
    
    
    active_row_id = active_cell['row'] if active_cell else None
    
    #dc = filtered_df['data_comment'][0]
    
    #print(dc)
    
    fig = go.Figure(
    data=[go.Scatter(x=dff['x'], y=dff['y'])],
    layout=go.Layout(
        #title=go.layout.Title(text="A Figure Specified By A Graph Object")
        ##title=go.layout.Title(text='y')
        title=go.layout.Title(text=filtered_df['data_label'].iloc[0])
        ##df_test['Btime'].iloc[0]
    )
)
    
    fig.update_xaxes(type="log")
    fig.update_yaxes(type="log")
    
    fig.update_layout(
    xaxis_title=filtered_df['x_units'].iloc[0],
    yaxis_title=filtered_df['y_units'].iloc[0],
    legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
    
    
    graph = html.Div(children=[dcc.Graph(figure=fig)])
    
    
    '''graph = html.Div(children=[dcc.Graph(
            #id=column + '--row-ids',
            figure={
                'data': [
                    {
                        'x': dff['x'],
                        'y': dff['y'],
                        'type': 'scatter'#,
                        #'marker': {'color': colors},
                    }
                ],
                'layout': {
                    'xaxis': {'automargin': True},
                    'yaxis': {
                        'automargin': True,
                        'title': 'y',
                    },
                    'height': 250,
                    'margin': {'t': 10, 'l': 10, 'r': 10},
                },
            },
        )])'''
        # check if column exists - user may have deleted it
        # If `column.deletable=False`, then you don't
        # need to do this check.
        #for column in ['pop', 'lifeExp', 'gdpPercap'] if column in dff
        #for column in ['data_values'] if column in dff
    
    return graph

In [ ]:
#app.run_server(debug=True,host='0.0.0.0', port=5051)

In [ ]:
#app.run_server(mode='jupyterlab', port = 5052, dev_tools_ui=True, #debug=True,
#              dev_tools_hot_reload =True, threaded=True)#

In [ ]:
#app.run_server(host="0.0.0.0", port=5051)

In [ ]:
from dash import dcc
import plotly.graph_objs as go

fig = go.Figure(data=[go.Scatter(x=[1, 2, 3], y=[4, 1, 2])])

graph = dcc.Graph(figure=fig)

layout10 =  html.Div(graph)


In [ ]:
#app.layout = layout10
#app.run_server(host="0.0.0.0", port=5051, debug=True, use_reloader=False) 

In [ ]:

row1 =  dbc.Row([dbc.Col(
            [
                dbc.Label("Result Type"),
                resulttype_dropdown,
            ],
            #width=2,
        ),
        dbc.Col(
            [
                dbc.Label("Spin"),
                spin_dropdown,
            ],
            #width=2,
        ),])
    

row2 = dbc.Row([dbc.Col(
            [
                dbc.Label("Experiment"),
                experiment_dropdown,
            ],
            #width=2,
        ),
        dbc.Col(
            [
               dbc.Label("Greatest Hits"),
                greatesthits_dropdown,
            ],
            #width=2,
        ),])
    
#form = dbc.Container(
#    children=[dbc.Row(
#    [
#        row1,
#        row2,        
#    ],
#    className="g-3",
#)])

form = html.Div(
    children=[dbc.Row(
    [
        row1,
        row2,        
    ], className="col", style={"height":"30h","width": "10vw", "padding":"0px"}
)])

In [ ]:
mainpanel = html.Div(
            [
                html.Div(
            [
            #html.H3(children='Left Full Height'),
                #graph,
                dcc.Graph(id="graph_out",style={'width': '80vh', 'height': '70vh'})
            ],
            style={"height":"70vh", "width": "50vw", "background-color":"purple", "padding":"0px"})
            ],
            className="col", style={"height":"70vh","width": "50vw", "padding":"0px"}##,"background-color":"purple"}
        )

rightrow1 =          html.Div(
            [
                html.Div(
            [
                ##html.H3(children='Panel 1'),
                form,
            ],
            className="short-div", style={"height":"35vh","background-color":"green"}
        ), 
                 html.Div(
            [
                html.H3(children='Panel 2'),
            ],
            className="short-div", style={"height":"35vh","background-color":"yellow"}
        ), 
            ],
            className="col", style={"width": "30vw","padding":"0px",}
        )

rightrow2 = html.Div(
            [
                html.Div(
            [
                html.H3(children='Panel 3'),
            ],
            className="short-div", style={"height":"35vh","background-color":"red"}
        ), 
                 html.Div(
            [
                html.H3(children='Panel 4'),
            ],
            className="short-div", style={"height":"35vh","background-color":"blue"}
        ), 
            ],
            className="col", style={"width": "30vw", "padding":"0px"}
        )

layout6 =  html.Div(className="row",children=[mainpanel,rightrow1,rightrow2,], style={"padding":"0px"})

In [ ]:

header_height, footer_height = "5vh", "5vh"
sidebar_width, adbar_width = "9vh", "9vh"
table_height = "20vh"
headerplustable_height = "25vh"

HEADER_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "right": 0,
    "height": header_height,
    "padding": "0rem 0rem",
    "background-color": "lightblue",
}

TABLE_STYLE = {
    "position": "fixed",
    "top": header_height,
    "margin-left": sidebar_width,
    "margin-right": adbar_width,
    "margin-bottom": footer_height,
    "left": 0,
    "right": 0,
    "height": table_height,
    "padding": "0rem 0rem",
    "background-color": "white",
}

SIDEBAR_STYLE = {
    "position": "fixed",
    "top": header_height,
    "left": 0,
    "bottom": footer_height,
    "width": sidebar_width,
    "padding": "0rem 0rem",
    "background-color": "lightgreen",
}

ADBAR_STYLE = {
    "position": "fixed",
    "top": header_height,
    "right": 0,
    "bottom": footer_height,
    "width": adbar_width,
    "padding": "0rem 0rem",
    "background-color": "lightgreen",
}

FOOTER_STYLE = {
    "position": "fixed",
    "bottom": 0,
    "left": 0,
    "right": 0,
    "height": footer_height,
    "padding": "0rem 0rem",
    "background-color": "gray",
}

CONTENT_STYLE = {
    "position": "fixed",
    "top": headerplustable_height,
    "height" : "68vh",
    #"margin-top": headerplustable_height,
    "margin-left": "10vh" , ## sidebar_width,
    "margin-right": "10vh", # adbar_width,
    "margin-bottom": footer_height,
    "padding": "0vh 0vh",
}

header = html.Div([
    html.P('Dashboard Template')], style=HEADER_STYLE,
)

fdivs = [html.P("ACG Data Engineering")]
footer = html.Div(fdivs, style=FOOTER_STYLE,)
#footer = html.Div(filters, style=FOOTER_STYLE,)

adbar = html.Div([
    html.P('R')],
    style=ADBAR_STYLE,
)

sidebar = html.Div(
    [html.P("L", className="lead")],
    #filters,
    style=SIDEBAR_STYLE,
)

#cdivs = [html.P("content")]
cdivs = [layout6]



content = html.Div(cdivs, style=CONTENT_STYLE)



In [ ]:
tabledivs = html.Div([
    data_table,
    #html.Div(id='datatable-row-ids-container')
])

#tabledivs = [html.P("Table")]
table = html.Div(tabledivs,style=TABLE_STYLE,)

In [ ]:
layout3 = html.Div([header, table, sidebar, adbar, content, footer])

In [ ]:
app.layout = layout3


@app.callback(
    #Output(component_id='datatable-row-ids-container', component_property='children'),
    Output('graph_out','figure'),
    Input('datatable-row-ids', 'derived_virtual_row_ids'),
    Input('datatable-row-ids', 'selected_row_ids'),
    Input('datatable-row-ids', 'active_cell'))
def update_graphs(row_ids, selected_row_ids, active_cell):
    # When the table is first rendered, `derived_virtual_data` and
    # `derived_virtual_selected_rows` will be `None`. This is due to an
    # idiosyncrasy in Dash (unsupplied properties are always None and Dash
    # calls the dependent callbacks when the component is first rendered).
    # So, if `rows` is `None`, then the component was just rendered
    # and its value will be the same as the component's dataframe.
    # Instead of setting `None` in here, you could also set
    # `derived_virtual_data=df.to_rows('dict')` when you initialize
    # the component.
    
    #derived_virtual_data=data_df.to_dict('records')
    
    #selected_id_set = set(selected_row_ids or [])
    
    #value_list = ["apple"]

    boolean_series = plot_df.id.isin(selected_row_ids)

    filtered_df = plot_df[boolean_series].copy()
    
    #plot_df[plot_df['id']==100]

    plot_data = datastring2dataframe(filtered_df.iloc[0]).copy()
    
    if row_ids is None:
        dff = plot_data
        # pandas Series works enough like a list for this to be OK
        #row_ids = filtered_df['id']
        row_ids = [0]
    else:
        #dff = data_df.loc[row_ids]
        dff = plot_data
    
    
    active_row_id = active_cell['row'] if active_cell else None
    
    #dc = filtered_df['data_comment'][0]
    
    #print(dc)
    
    fig = go.Figure(
    data=[go.Scatter(x=dff['x'], y=dff['y'])],
    layout=go.Layout(
        #title=go.layout.Title(text="A Figure Specified By A Graph Object")
        ##title=go.layout.Title(text='y')
        title=go.layout.Title(text=filtered_df['data_label'].iloc[0])
        ##df_test['Btime'].iloc[0]
    )
)
    
    fig.update_xaxes(type="log")
    fig.update_yaxes(type="log")
    
    fig.update_layout(
    xaxis_title=filtered_df['x_units'].iloc[0],
    yaxis_title=filtered_df['y_units'].iloc[0],
    legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
    
    
    #graph = html.Div(children=[dcc.Graph(figure=fig)])
    
    
    '''graph = html.Div(children=[dcc.Graph(
            #id=column + '--row-ids',
            figure={
                'data': [
                    {
                        'x': dff['x'],
                        'y': dff['y'],
                        'type': 'scatter'#,
                        #'marker': {'color': colors},
                    }
                ],
                'layout': {
                    'xaxis': {'automargin': True},
                    'yaxis': {
                        'automargin': True,
                        'title': 'y',
                    },
                    'height': 250,
                    'margin': {'t': 10, 'l': 10, 'r': 10},
                },
            },
        )])'''
        # check if column exists - user may have deleted it
        # If `column.deletable=False`, then you don't
        # need to do this check.
        #for column in ['pop', 'lifeExp', 'gdpPercap'] if column in dff
        #for column in ['data_values'] if column in dff
    
    return fig


In [ ]:

#app.config.suppress_callback_exceptions = True
app.run_server(host="0.0.0.0", port=5051, debug=True, use_reloader=False) 